# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB
df = pd.read_csv('TrailMakeB_with_MethylomeDetails.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\TrailMakeB
(216, 61926)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Trail_Making_B_Category,cg02315971,cg24133836,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.030109,0.016106,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.046350,0.904242,0.021033,0.813738
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.028697,0.017000,...,0.109870,0.056582,0.064137,0.053802,0.083556,0.042638,0.041733,0.896521,0.033285,0.753096
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.025308,0.017806,...,0.179598,0.037220,0.050652,0.027469,0.056702,0.032307,0.024831,0.924415,0.016215,0.900643


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','Trail_Making_B_Category','Age','AgeCate'], axis=1)
df1.head(2)

,APPRDX,cg02315971,cg24133836,cg03286969,cg11738782,cg25741731,cg24629744,cg23836745,cg11685489,cg06081147,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,2,0.030109,0.016106,0.019371,0.884151,0.026326,0.868373,0.905809,0.825562,0.691096,...,0.148515,0.036822,0.053052,0.044051,0.040994,0.047501,0.046350,0.904242,0.021033,0.813738
1,1,0.028697,0.017000,0.018631,0.871526,0.026148,0.866396,0.922285,0.881489,0.647436,...,0.109870,0.056582,0.064137,0.053802,0.083556,0.042638,0.041733,0.896521,0.033285,0.753096


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.4666666666666666
2
Mean accuracy: 0.5000000000000001
3
Mean accuracy: 0.5416666666666666
4
Mean accuracy: 0.5833333333333333
5
Mean accuracy: 0.5416666666666666
6
Mean accuracy: 0.4666666666666667
7
Mean accuracy: 0.5
8
Mean accuracy: 0.425
9
Mean accuracy: 0.525
10
Mean accuracy: 0.5
11
Mean accuracy: 0.5083333333333333
12
Mean accuracy: 0.5
13
Mean accuracy: 0.55
14
Mean accuracy: 0.4333333333333333
15
Mean accuracy: 0.5333333333333333
16
Mean accuracy: 0.45
17
Mean accuracy: 0.4333333333333333
18
Mean accuracy: 0.4833333333333333
19
Mean accuracy: 0.425
20
Mean accuracy: 0.5
21
Mean accuracy: 0.39166666666666666
22
Mean accuracy: 0.5333333333333334
23
Mean accuracy: 0.5833333333333334
24
Mean accuracy: 0.45
25
Mean accuracy: 0.55
26
Mean accuracy: 0.5000000000000001
27
Mean accuracy: 0.4833333333333332
28
Mean accuracy: 0.5249999999999999
29
Mean accuracy: 0.5333333333333334
30
Mean accuracy: 0.475
31
Mean accuracy: 0.5250000000000001
32
Mean accuracy: 0.516666666

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_AllCpG_100RF.csv", index = False)

0.4982500000000002


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5416666666666667
2
Mean accuracy: 0.4083333333333333
3
Mean accuracy: 0.4916666666666666
4
Mean accuracy: 0.5833333333333333
5
Mean accuracy: 0.5333333333333334
6
Mean accuracy: 0.5333333333333333
7
Mean accuracy: 0.5833333333333333
8
Mean accuracy: 0.525
9
Mean accuracy: 0.6333333333333333
10
Mean accuracy: 0.6
11
Mean accuracy: 0.4750000000000001
12
Mean accuracy: 0.41666666666666663
13
Mean accuracy: 0.5333333333333333
14
Mean accuracy: 0.4083333333333334
15
Mean accuracy: 0.4916666666666667
16
Mean accuracy: 0.4916666666666667
17
Mean accuracy: 0.55
18
Mean accuracy: 0.5
19
Mean accuracy: 0.5083333333333333
20
Mean accuracy: 0.5
21
Mean accuracy: 0.4916666666666667
22
Mean accuracy: 0.4916666666666666
23
Mean accuracy: 0.5166666666666667
24
Mean accuracy: 0.5666666666666667
25
Mean accuracy: 0.5166666666666667
26
Mean accuracy: 0.475
27
Mean accuracy: 0.4916666666666667
28
Mean accuracy: 0.5833333333333334
29
Mean accuracy: 0.45833333333333337
30
Mean accuracy: 0

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_Rand50CpG_100RF.csv", index = False)

0.5110000000000002


## RanFor 100 times. Samples = 97

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(97, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5166666666666667
2
Mean accuracy: 0.4833333333333333
3
Mean accuracy: 0.5333333333333334
4
Mean accuracy: 0.5416666666666667
5
Mean accuracy: 0.5583333333333333
6
Mean accuracy: 0.4916666666666667
7
Mean accuracy: 0.5333333333333333
8
Mean accuracy: 0.5083333333333333
9
Mean accuracy: 0.6083333333333333
10
Mean accuracy: 0.5083333333333333
11
Mean accuracy: 0.4666666666666667
12
Mean accuracy: 0.4833333333333333
13
Mean accuracy: 0.55
14
Mean accuracy: 0.375
15
Mean accuracy: 0.4999999999999999
16
Mean accuracy: 0.5666666666666667
17
Mean accuracy: 0.4916666666666667
18
Mean accuracy: 0.4333333333333334
19
Mean accuracy: 0.5416666666666666
20
Mean accuracy: 0.575
21
Mean accuracy: 0.4333333333333334
22
Mean accuracy: 0.425
23
Mean accuracy: 0.4833333333333334
24
Mean accuracy: 0.5666666666666667
25
Mean accuracy: 0.5
26
Mean accuracy: 0.4666666666666666
27
Mean accuracy: 0.4916666666666666
28
Mean accuracy: 0.5333333333333333
29
Mean accuracy: 0.4583333333333333
30
M

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_Rand97CpG_100RF.csv", index = False)

0.5026666666666666


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('TrailMakeB_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(216, 52)


,PATNO,APPRDX,cg18439474,cg26354221,cg01746044,cg17898737,cg12153529,cg02109003,cg13122031,cg20721322,...,cg24421842,cg15469205,cg09495953,cg16620860,cg19810426,cg10425946,cg12417702,cg27497687,cg16356224,cg25911279
0,3000,2,0.790590,0.918085,0.021957,0.923107,0.026793,0.717723,0.910175,0.255404,...,0.790974,0.656929,0.812500,0.624056,0.337330,0.017390,0.069469,0.037778,0.148380,0.035425
1,3001,1,0.818168,0.924711,0.051224,0.916143,0.032767,0.639496,0.813401,0.208455,...,0.821622,0.865868,0.907085,0.582799,0.293279,0.051054,0.030323,0.033875,0.179434,0.031463


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg18439474,cg26354221,cg01746044,cg17898737,cg12153529,cg02109003,cg13122031,cg20721322,ch.22.138449R,...,cg24421842,cg15469205,cg09495953,cg16620860,cg19810426,cg10425946,cg12417702,cg27497687,cg16356224,cg25911279
0,2,0.790590,0.918085,0.021957,0.923107,0.026793,0.717723,0.910175,0.255404,0.070647,...,0.790974,0.656929,0.812500,0.624056,0.337330,0.017390,0.069469,0.037778,0.148380,0.035425
1,1,0.818168,0.924711,0.051224,0.916143,0.032767,0.639496,0.813401,0.208455,0.076957,...,0.821622,0.865868,0.907085,0.582799,0.293279,0.051054,0.030323,0.033875,0.179434,0.031463


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7
2
Mean accuracy: 0.6666666666666667
3
Mean accuracy: 0.675
4
Mean accuracy: 0.6916666666666667
5
Mean accuracy: 0.7666666666666666
6
Mean accuracy: 0.7166666666666667
7
Mean accuracy: 0.7
8
Mean accuracy: 0.7333333333333333
9
Mean accuracy: 0.7166666666666667
10
Mean accuracy: 0.7416666666666667
11
Mean accuracy: 0.6416666666666666
12
Mean accuracy: 0.675
13
Mean accuracy: 0.675
14
Mean accuracy: 0.7083333333333333
15
Mean accuracy: 0.7416666666666667
16
Mean accuracy: 0.7416666666666668
17
Mean accuracy: 0.7416666666666666
18
Mean accuracy: 0.675
19
Mean accuracy: 0.6583333333333334
20
Mean accuracy: 0.6749999999999999
21
Mean accuracy: 0.7
22
Mean accuracy: 0.7083333333333334
23
Mean accuracy: 0.6833333333333333
24
Mean accuracy: 0.7500000000000001
25
Mean accuracy: 0.75
26
Mean accuracy: 0.7083333333333333
27
Mean accuracy: 0.7833333333333333
28
Mean accuracy: 0.6583333333333333
29
Mean accuracy: 0.6916666666666667
30
Mean accuracy: 0.7333333333333334
31
Mean ac

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_Boruta50CpG_100RF.csv", index = False)

0.7139166666666666


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Top50NMI_TrailMakeB_Data.csv')
print(df.shape)
df.head(2)

(216, 52)


,PATNO,APPRDX,cg04958915,cg03953298,cg23983141,cg18050804,cg05356336,cg16850288,cg19245310,cg11212801,...,cg03128167,cg10644659,cg08458872,cg12449007,cg21020710,cg20502123,cg23249278,cg16021428,cg00156001,cg09722742
0,3000,2,0.953291,0.835808,0.873134,0.903780,0.730565,0.861231,0.838944,0.940099,...,0.967504,0.892376,0.917712,0.920729,0.064918,0.832224,0.882236,0.059132,0.913846,0.022733
1,3001,1,0.912131,0.854043,0.863465,0.862122,0.794499,0.830251,0.827915,0.920422,...,0.958426,0.889949,0.916094,0.905869,0.078724,0.861767,0.886454,0.021360,0.883715,0.020150


In [16]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg04958915,cg03953298,cg23983141,cg18050804,cg05356336,cg16850288,cg19245310,cg11212801,cg23177739,...,cg03128167,cg10644659,cg08458872,cg12449007,cg21020710,cg20502123,cg23249278,cg16021428,cg00156001,cg09722742
0,2,0.953291,0.835808,0.873134,0.903780,0.730565,0.861231,0.838944,0.940099,0.710892,...,0.967504,0.892376,0.917712,0.920729,0.064918,0.832224,0.882236,0.059132,0.913846,0.022733
1,1,0.912131,0.854043,0.863465,0.862122,0.794499,0.830251,0.827915,0.920422,0.597176,...,0.958426,0.889949,0.916094,0.905869,0.078724,0.861767,0.886454,0.021360,0.883715,0.020150


In [17]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.775
2
Mean accuracy: 0.8
3
Mean accuracy: 0.7416666666666667
4
Mean accuracy: 0.7333333333333334
5
Mean accuracy: 0.6583333333333334
6
Mean accuracy: 0.7416666666666668
7
Mean accuracy: 0.7333333333333333
8
Mean accuracy: 0.7833333333333333
9
Mean accuracy: 0.7333333333333333
10
Mean accuracy: 0.7333333333333334
11
Mean accuracy: 0.7
12
Mean accuracy: 0.7666666666666667
13
Mean accuracy: 0.7666666666666666
14
Mean accuracy: 0.7500000000000001
15
Mean accuracy: 0.7416666666666666
16
Mean accuracy: 0.775
17
Mean accuracy: 0.7
18
Mean accuracy: 0.7666666666666666
19
Mean accuracy: 0.725
20
Mean accuracy: 0.7416666666666667
21
Mean accuracy: 0.7083333333333333
22
Mean accuracy: 0.7666666666666667
23
Mean accuracy: 0.7583333333333334
24
Mean accuracy: 0.6916666666666667
25
Mean accuracy: 0.7583333333333332
26
Mean accuracy: 0.7583333333333333
27
Mean accuracy: 0.725
28
Mean accuracy: 0.7416666666666667
29
Mean accuracy: 0.7583333333333333
30
Mean accuracy: 0.7416666666666

In [18]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_NMI50CpG_100RF.csv", index = False)

0.7474166666666668


## RanFor Top 50 Boruta + NMI 100 times. Samples = 97

In [19]:
df = pd.read_csv('TrailMakeB_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(216, 99)


,PATNO,APPRDX,cg04958915,cg03953298,cg23983141,cg18050804,cg05356336,cg16850288,cg19245310,cg11212801,...,cg16476940,cg24421842,cg15469205,cg09495953,cg16620860,cg19810426,cg10425946,cg12417702,cg27497687,cg16356224
0,3000,2,0.953291,0.835808,0.873134,0.903780,0.730565,0.861231,0.838944,0.940099,...,0.804358,0.790974,0.656929,0.812500,0.624056,0.337330,0.017390,0.069469,0.037778,0.148380
1,3001,1,0.912131,0.854043,0.863465,0.862122,0.794499,0.830251,0.827915,0.920422,...,0.801166,0.821622,0.865868,0.907085,0.582799,0.293279,0.051054,0.030323,0.033875,0.179434


In [20]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg04958915,cg03953298,cg23983141,cg18050804,cg05356336,cg16850288,cg19245310,cg11212801,cg23177739,...,cg16476940,cg24421842,cg15469205,cg09495953,cg16620860,cg19810426,cg10425946,cg12417702,cg27497687,cg16356224
0,2,0.953291,0.835808,0.873134,0.903780,0.730565,0.861231,0.838944,0.940099,0.710892,...,0.804358,0.790974,0.656929,0.812500,0.624056,0.337330,0.017390,0.069469,0.037778,0.148380
1,1,0.912131,0.854043,0.863465,0.862122,0.794499,0.830251,0.827915,0.920422,0.597176,...,0.801166,0.821622,0.865868,0.907085,0.582799,0.293279,0.051054,0.030323,0.033875,0.179434


In [21]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7166666666666667
2
Mean accuracy: 0.7666666666666667
3
Mean accuracy: 0.7416666666666667
4
Mean accuracy: 0.7583333333333334
5
Mean accuracy: 0.7
6
Mean accuracy: 0.725
7
Mean accuracy: 0.7
8
Mean accuracy: 0.7250000000000001
9
Mean accuracy: 0.7583333333333334
10
Mean accuracy: 0.7166666666666667
11
Mean accuracy: 0.775
12
Mean accuracy: 0.7666666666666668
13
Mean accuracy: 0.7333333333333333
14
Mean accuracy: 0.7916666666666666
15
Mean accuracy: 0.7833333333333333
16
Mean accuracy: 0.8
17
Mean accuracy: 0.7666666666666667
18
Mean accuracy: 0.75
19
Mean accuracy: 0.7166666666666667
20
Mean accuracy: 0.7333333333333332
21
Mean accuracy: 0.7083333333333333
22
Mean accuracy: 0.7583333333333333
23
Mean accuracy: 0.7416666666666666
24
Mean accuracy: 0.75
25
Mean accuracy: 0.75
26
Mean accuracy: 0.7416666666666667
27
Mean accuracy: 0.7916666666666666
28
Mean accuracy: 0.7333333333333333
29
Mean accuracy: 0.7499999999999999
30
Mean accuracy: 0.7416666666666666
31
Mean accu

In [22]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("TrailMakeB_Top50sCpG97_100RF.csv", index = False)

0.7545833333333332
